In [10]:
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [19]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/admin/Documents/Projects/AIFootballPredictions/data/raw/SP1_merged.csv')

# Display basic information about the DataFrame
print(df.head())
print(df.shape)
print(df.info())

   Div        Date   Time    HomeTeam   AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
0  SP1  15/08/2025  18:00      Girona  Vallecano     1     3   A     0     3   
1  SP1  15/08/2025  20:30  Villarreal     Oviedo     2     0   H     2     0   
2  SP1  16/08/2025  18:30    Mallorca  Barcelona     0     3   A     0     2   
3  SP1  16/08/2025  20:30      Alaves    Levante     2     1   H     1     0   
4  SP1  16/08/2025  20:30    Valencia   Sociedad     1     1   D     0     0   

  HTR  HS  AS  HST  AST  HF  AF  HC  AC  HY  AY  HR  AR  B365H  B365D  B365A  \
0   A   7  16    2    5   8  17   2   4   0   1   1   0   2.25   3.25    3.3   
1   H  25   5   10    4  10   8  10   1   1   2   0   1   1.40   4.75    8.0   
2   A   4  24    1    8   8  17   3   6   4   1   2   0   7.00   5.00    1.4   
3   H  15   7    4    3  15  13  10   1   0   1   0   0   2.15   3.00    3.8   
4   D  18  13    2    3   5  11   8   7   1   2   0   0   2.60   2.90    3.1   

    BWH   BWD   BWA   PSH   PSD   PSA 

In [13]:
# Count missing values per column and total
missing_per_col = df.isnull().sum()
total_missing = missing_per_col.sum()
missing_percent = (missing_per_col / len(df) * 100).round(2)

print(f"Total missing values: {total_missing}\n")

print("Missing values per column (only columns with >0 missing, sorted):")
print(missing_per_col[missing_per_col > 0].sort_values(ascending=False))

print("\nMissing percent per column (only columns with >0 missing, sorted):")
print(missing_percent[missing_percent > 0].sort_values(ascending=False))

# Rows with any missing value
rows_with_missing = df.isnull().any(axis=1).sum()
print(f"\nRows with at least one missing value: {rows_with_missing} / {len(df)}")

Total missing values: 971

Missing values per column (only columns with >0 missing, sorted):
BWH       160
BWD       160
BWA       160
BWCH      159
BWCD      159
BWCA      159
P>2.5       4
P<2.5       4
PC>2.5      2
PC<2.5      2
PAHH        1
PAHA        1
dtype: int64

Missing percent per column (only columns with >0 missing, sorted):
BWH       12.50
BWD       12.50
BWA       12.50
BWCH      12.42
BWCD      12.42
BWCA      12.42
P>2.5      0.31
P<2.5      0.31
PC>2.5     0.16
PC<2.5     0.16
PAHH       0.08
PAHA       0.08
dtype: float64

Rows with at least one missing value: 166 / 1280


In [14]:
# convert Date to datetime (dayfirst) and sort by most recent date (and time if present)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

if 'Time' in df.columns:
  # combine Date and Time for precise sorting; handle missing/invalid times
  df['DateTime'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time'].astype(str), errors='coerce')
  df_sorted = df.sort_values('DateTime', ascending=False).reset_index(drop=True)
else:
  df_sorted = df.sort_values('Date', ascending=False).reset_index(drop=True)

df_sorted.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,PSH,PSD,PSA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,PSCH,PSCD,PSCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,DateTime
0,SP1,2025-12-01,20:00,Vallecano,Valencia,1,1,D,1,0,H,19,10,7,2,11,9,10,5,1,1,0,0,1.95,3.40,4.10,1.95,3.40,4.00,1.98,3.41,4.25,1.95,3.40,4.60,1.92,3.30,4.11,2.10,1.73,2.17,1.74,2.14,1.73,2.08,1.70,-0.50,1.98,1.88,1.99,1.92,1.98,1.93,1.90,1.87,2.00,3.30,3.90,2.05,3.20,3.90,2.11,3.21,4.11,2.07,3.30,4.00,2.04,3.18,3.94,2.20,1.67,2.29,1.68,2.23,1.67,2.19,1.65,-0.5,2.03,1.83,2.12,1.83,2.05,1.93,1.95,1.82,2025-12-01 20:00:00
1,SP1,2025-11-30,20:00,Girona,Real Madrid,1,1,D,1,0,H,10,25,4,4,9,13,2,5,3,0,0,0,6.50,5.25,1.40,6.50,5.25,1.41,6.99,5.28,1.42,7.50,5.25,1.43,6.60,5.05,1.40,1.40,3.00,1.40,3.01,1.41,3.00,1.39,2.87,1.25,1.98,1.88,2.01,1.90,2.00,22.00,1.93,4.35,7.00,5.50,1.38,6.75,5.25,1.39,7.01,5.71,1.38,8.00,5.75,1.40,7.19,5.42,1.36,1.36,3.20,1.37,3.18,1.36,3.30,1.35,3.08,1.5,1.88,1.98,1.92,2.01,1.90,1.98,1.85,1.91,2025-11-30 20:00:00
2,SP1,2025-11-30,17:30,Celta,Espanol,0,1,A,0,0,D,11,9,4,2,12,15,4,3,1,2,0,0,2.05,3.50,3.50,2.00,3.40,3.80,2.05,3.42,3.95,2.05,3.55,4.00,1.98,3.42,3.73,1.80,2.00,1.91,1.98,1.91,2.00,1.85,1.89,-0.50,2.05,1.75,2.06,1.85,2.05,1.86,1.96,1.80,1.85,3.50,4.33,1.90,3.40,4.20,1.92,3.57,4.40,1.92,3.50,4.33,1.89,3.43,4.20,1.80,2.00,1.99,1.92,1.96,2.00,1.90,1.86,-0.5,1.90,1.95,1.93,2.00,1.95,1.95,1.88,1.89,2025-11-30 17:30:00
3,SP1,2025-11-30,15:15,Sevilla,Betis,0,2,A,0,0,D,9,9,3,4,14,22,6,4,2,3,1,0,3.00,3.30,2.38,2.90,3.40,2.37,2.97,3.38,2.49,3.05,3.50,2.44,2.90,3.34,2.35,1.91,1.91,2.02,1.87,1.95,1.91,1.90,1.83,0.25,1.80,2.05,1.78,2.14,1.84,22.00,1.78,4.16,2.63,3.25,2.70,2.75,3.25,2.60,2.78,3.34,2.72,2.76,3.30,2.90,2.69,3.22,2.66,2.00,1.80,2.08,1.83,2.05,1.80,1.98,1.78,0.0,1.93,1.93,1.98,1.94,1.95,2.05,1.87,1.89,2025-11-30 15:15:00
4,SP1,2025-11-30,13:00,Sociedad,Villarreal,2,3,A,0,1,A,18,10,7,5,15,18,8,2,3,7,0,0,2.80,3.40,2.45,2.80,3.40,2.45,2.92,3.38,2.53,2.90,3.40,2.50,2.81,3.35,2.43,1.88,1.98,1.91,1.98,1.91,1.98,1.84,1.90,0.00,2.05,1.80,2.10,1.82,2.05,22.00,2.02,5.12,2.80,3.40,2.45,2.70,3.30,2.60,2.90,3.41,2.57,3.00,3.40,2.60,2.82,3.28,2.50,1.83,2.03,1.93,1.98,1.91,2.03,1.84,1.92,0.0,2.05,1.80,2.09,1.85,2.05,1.85,1.99,1.78,2025-11-30 13:00:00


In [15]:
column_names = df.columns.tolist()
print(column_names)

['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'PSH', 'PSD', 'PSA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'PSCH', 'PSCD', 'PSCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'DateTime']


In [16]:
# Display column names with their data types
df_sorted.dtypes

Div                  object
Date         datetime64[ns]
Time                 object
HomeTeam             object
AwayTeam             object
FTHG                  int64
FTAG                  int64
FTR                  object
HTHG                  int64
HTAG                  int64
HTR                  object
HS                    int64
AS                    int64
HST                   int64
AST                   int64
HF                    int64
AF                    int64
HC                    int64
AC                    int64
HY                    int64
AY                    int64
HR                    int64
AR                    int64
B365H               float64
B365D               float64
B365A               float64
BWH                 float64
BWD                 float64
BWA                 float64
PSH                 float64
PSD                 float64
PSA                 float64
MaxH                float64
MaxD                float64
MaxA                float64
AvgH                